# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [2]:
import requests

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

payload = {
    "model": "gpt-5-nano",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}

payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [3]:
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=payload
)

response.json()

{'id': 'chatcmpl-CX2cu5dNMNzhfJqudlJpfWqVMqtQj',
 'object': 'chat.completion',
 'created': 1761990956,
 'model': 'gpt-5-nano-2025-08-07',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "Here's a fun fact: A group of flamingos is called a flamboyance. And they get their pink color from the carotenoids in their diet. Want another fun fact on a specific topic?",
    'refusal': None,
    'annotations': []},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 11,
  'completion_tokens': 434,
  'total_tokens': 445,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 384,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': None}

In [4]:
response.json()["choices"][0]["message"]["content"]

"Here's a fun fact: A group of flamingos is called a flamboyance. And they get their pink color from the carotenoids in their diet. Want another fun fact on a specific topic?"

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [5]:
# Create OpenAI client

from openai import OpenAI
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content



'Fun fact: A day on Venus is longer than a year on Venus. It takes about 243 Earth days to rotate once, but only about 225 days to orbit the Sun. Want another fun fact?'

## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

In [6]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!


In [23]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

response = gemini.chat.completions.create(model="gemini-2.5-pro", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}]

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [13]:
print(requests.get("http://localhost:11434").content)
#get olama version
response = requests.get("http://localhost:11434/api/version")
print(response.json())
#ollama model running
response = requests.get("http://localhost:11434/api/ps")
print(response.json())

b'Ollama is running'
{'version': '0.12.0'}
{'models': []}


### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [15]:
!ollama pull llama3.2:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏  78 KB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏ 628 KB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏ 950 KB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏ 1.1 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏ 1.5 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏ 2.3 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏ 2.7 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏ 3.4 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏ 4.3 MB

In [17]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


In [18]:
# Get a fun fact

response = ollama.chat.completions.create(model="llama3.2:1b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'Here\'s a fun fact: Did you know that there is a type of jellyfish that is immortal? The Turritopsis dohrnii, also known as the "immortal jellyfish," is a species of jellyfish that can transform its body into a younger state through a process called transdifferentiation. This means that it can essentially revert back to its polyp stage, which is the juvenile form of a jellyfish, and then grow back into an adult again. This process can be repeated indefinitely, making the Turritopsis dohrnii theoretically immortal!'

In [19]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud

!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏  25 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 614 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 735 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 1.2 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 1.5 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 2.0 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 2.6 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 3.4 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 4.6 MB

In [20]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'<think>\n\n</think>\n\nSure! Here\'s a fun fact for you: **"The number 4 is the only square number that is exactly half of another square number."** \n\nLet me explain. Square numbers are numbers like 1, 4, 9, 16, etc., which are squares of integers. Now, if we multiply 4 by any other square number and square it again, we might end up with a number that is twice the original. For example:\n\n- \\( (2 \\times 4)^2 = 8^2 = 64 \\)  \n  While 4 itself squared is 16.\n\nSo, in this case:\n\\[ (2 \\times n^2)^2 = 4n^2 = 2(n^4) \\]\n\nBut wait, that might not be the best way to put it. The point is that among square numbers, only 4 fits this unique property.'

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [22]:
# Step 1: Create your prompts
system_prompt = """
You are a thoughful assistant who go deep on the topic and provide a detailed and crisp answer based on data 
and trends and has an expertise in small and medium business in Indian and especially bihar context.
"""
user_prompt = """
    I want to start a small or a medium scale business and have inital capital of 1000000 INR but capital won't be a problem
    as I can take loan from bank and other financial institutions. I want to set-up a business which will be consumption based so that it can sustain
    odd circumstances like covid or any other pandemic and looking for a net profit of atleast 50000 INR per month. I am ready to put extra efforts and can invest 12-14 hrs per day.
"""

# Step 2: Make the messages list

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Step 3: Call OpenAI
response = ollama.chat.completions.create(
    model="llama3.2:1b",
    messages=messages
)

# Step 4: print the result
print(response.choices[0].message.content)

With an initial capital of 1,000,000 INR, the possibilities are vast, and setting up a sustainable business that provides a decent margin for net profit is feasible. To help you achieve your goals:

### **Business Idea Selection**

Considering your focus on consumption-based businesses and considering the Indian market's diverse consumer base, here are a few potential business ideas that could be viable with your capital:

1.  **E-commerce Store (B2C)**: Focus on niche markets where products cater to specific consumers. You can leverage e-commerce platforms like Amazon, Flipkart, or Zeepush.
2.  **Meal Kit Delivery (Food delivery and preparation services)**: Partnering with local restaurants or food outlets, using their kitchen facilities, and providing meal kits could be a profitable business model.
3.  **Grocery Products Manufacturing**: If you have expertise in manufacturing, consider producing value-added products like organic or ready-to-prepare meals through your own factory.
4. 